# Creacion del modelo de regresion lineal multiple

En este notebook nos centraremos en crear el modelo de regresion lineal multiple y evaluar sus metricas 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import statsmodels.api as sm

df_laptops = pd.read_csv("data/df_model_ready.csv")

In [2]:
# Definimos las variables predictoras y la variable objetivo
# Variables numéricas ya transformadas con Box-Cox
numerical_boxcox_features = [
    'Inches_BoxCox', 'Ram_BoxCox', 'Weight_BoxCox', 'CPU_freq_BoxCox',
    'PrimaryStorage_BoxCox', 'SecondaryStorage_BoxCox', 'ScreenPixels_BoxCox'
]

# Variables binarias
binary_features = [
    'Touchscreen', 'IPSpanel', 'RetinaDisplay'
]

# Variables categóricas
categorical_features = [
    'Company', 'TypeName', 'OS', 'PrimaryStorageType',
    'SecondaryStorageType', 'CPU_company', 'GPU_company'
]

# Variable objetivo
target_variable = 'Price_euros_BoxCox'

# Combinamos todas las características predictoras
all_features = numerical_boxcox_features + binary_features + categorical_features

# Manejamos las variables categóricas con One-Hot Encoding
print("\nAplicando One-Hot Encoding a las variables categóricas...")
df_encoded = pd.get_dummies(df_laptops, columns=categorical_features, drop_first=True) # drop_first para evitar multicolinealidad perfecta
print(f"Dimensiones del DataFrame después de One-Hot Encoding: {df_encoded.shape}")

# Separamos X (features) e y (target)
X = df_encoded.drop(columns=[target_variable])
y = df_encoded[target_variable]

# Aseguramos de que no haya columnas con todos ceros o nulos después del encoding si drop_first=False
# (aunque drop_first=True ya ayuda a mitigar esto para la multicolinealidad perfecta)
X = X.loc[:, (X != 0).any(axis=0)] # Eliminar columnas si todas son cero
X = X.dropna(axis=1) # Eliminar columnas si contienen NaN (raro después de get_dummies si la entrada no tenía NaN)

print(f"\nNúmero de características finales para el modelo: {X.shape[1]}")

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} muestras")

# Entrenamos un modelo de Regresión Lineal Múltiple
print("\nEntrenando el modelo de Regresión Lineal Múltiple...")
model = LinearRegression()
model.fit(X_train, y_train)
print("Modelo entrenado exitosamente.")


Aplicando One-Hot Encoding a las variables categóricas...
Dimensiones del DataFrame después de One-Hot Encoding: (1275, 53)

Número de características finales para el modelo: 52
Tamaño del conjunto de entrenamiento: 1020 muestras
Tamaño del conjunto de prueba: 255 muestras

Entrenando el modelo de Regresión Lineal Múltiple...
Modelo entrenado exitosamente.


In [3]:
# Evaluamos el rendimiento del modelo en el conjunto de ENTRENAMIENTO 
y_pred_train = model.predict(X_train)

r2_train = r2_score(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)

n_train = len(y_train)
k_train = X_train.shape[1] # Número de variables predictoras en el conjunto de entrenamiento

r2_adjusted_train = np.nan # Inicializamos a NaN por si no se puede calcular
if n_train - k_train - 1 > 0:
    r2_adjusted_train = 1 - ((1 - r2_train) * (n_train - 1)) / (n_train - k_train - 1)
else:
    print("Advertencia: No se puede calcular el R-cuadrado ajustado para entrenamiento (n - k - 1 <= 0).")


# Evaluamos el rendimiento del modelo en el conjunto de PRUEBA ---
y_pred_test = model.predict(X_test)

r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)

n_test = len(y_test)
k_test = X_test.shape[1]

r2_adjusted_test = np.nan # Inicializamos a NaN por si no se puede calcular
if n_test - k_test - 1 > 0:
    r2_adjusted_test = 1 - ((1 - r2_test) * (n_test - 1)) / (n_test - k_test - 1)
else:
    print("Advertencia: No se puede calcular el R-cuadrado ajustado para prueba (n - k - 1 <= 0).")


# --- Imprimimos la tabla de métricas en formato Markdown con alineación mejorada ---
print("\n# Métricas de Evaluación del Modelo de Regresión Lineal\n")
print("| Métrica                                  | Entrenamiento | Prueba        |")
print("| :--------------------------------------- | ------------: | ------------: |") # Alineamos métricas a la izquierda, valores a la derecha

# Usamos f-strings con padding para asegurar la alineación de las columnas:
# {:<40} -> Columna de Métrica: 40 caracteres de ancho, alineada a la izquierda
# {:>13.4f} -> Columnas de valores: 13 caracteres de ancho, alineada a la derecha, con 4 decimales
print(f"| {'R-cuadrado (R²)':<40} | {r2_train:>13.4f} | {r2_test:>13.4f} |")
print(f"| {'R-cuadrado Ajustado (R²_adj)':<40} | {r2_adjusted_train:>13.4f} | {r2_adjusted_test:>13.4f} |")
print(f"| {'Error Absoluto Medio (MAE)':<40} | {mae_train:>13.4f} | {mae_test:>13.4f} |")
print(f"| {'Error Cuadrático Medio (MSE)':<40} | {mse_train:>13.4f} | {mse_test:>13.4f} |")
print(f"| {'Raíz del Error Cuadrático Medio (RMSE)':<40} | {rmse_train:>13.4f} | {rmse_test:>13.4f} |")


# Métricas de Evaluación del Modelo de Regresión Lineal

| Métrica                                  | Entrenamiento | Prueba        |
| :--------------------------------------- | ------------: | ------------: |
| R-cuadrado (R²)                          |        0.8365 |        0.8094 |
| R-cuadrado Ajustado (R²_adj)             |        0.8277 |        0.7603 |
| Error Absoluto Medio (MAE)               |        0.4394 |        0.4717 |
| Error Cuadrático Medio (MSE)             |        0.3195 |        0.3297 |
| Raíz del Error Cuadrático Medio (RMSE)   |        0.5652 |        0.5742 |


Gracias a la tabla anterior si bien el $R^2$ y el $R^2$ ajutado son altos quiero encotrar la forma de ver si puedo incrementar el valor de los mismos esto lo veremos en el siguiente notebook

